In [1]:
import numpy as np
import pandas as pd
import smatch

#If using script on terminal
#from tqdm import tqdm
from tqdm.notebook import tqdm

In [2]:
data = pd.read_csv('SDSS_catalog_hpix.csv')
print(len(data))
data.head()

41844


,recno,objID,M15,Pdisk,Pedgeon,PbarGZ2,PbarNair10,Pmerg,Pbulge,Pcigar,TType,PS0,RA,DE,HPIX_4096
0,562371,588015509289107593,562371,0.007895,8.908050e-03,0.004785,0.416281,0.025819,0.003938,0.284150,3.99326,0.999085,45.0749,0.11085,73
1,562372,588015509289107598,562372,0.000452,4.362040e-07,0.002019,0.348180,0.041967,0.079616,0.000327,1.98614,0.993579,45.0906,0.13660,79
2,562373,588015509289107620,562373,0.002187,9.806280e-01,0.009124,0.320715,0.019539,0.905458,0.999917,1.22539,0.973403,45.1526,0.17028,264
3,149629,587731513154273388,149629,0.002267,8.160070e-01,0.004812,0.288495,0.006018,0.000113,0.984632,3.29565,0.987963,45.2996,0.30291,345
4,149628,587731513154207938,149628,0.000280,7.518660e-02,0.054535,0.312281,0.084383,0.095057,0.006318,-0.38629,0.930315,45.2152,0.35570,466


In [3]:
catalog = pd.read_csv('des_matched.csv')
print(len(catalog))
print(catalog.columns)
catalog.head()

491186
Index(['COADD_OBJECT_ID', 'TILENAME', 'HPIX_16384', 'HPIX_4096',
       'DNF_ZMEAN_SOF', 'RA', 'DEC', 'FLUX_RADIUS_G', 'FLUX_RADIUS_R',
       'FLUX_RADIUS_I', 'FLUX_RADIUS_Z', 'KRON_RADIUS', 'GAP_FLUX_G',
       'MOF_BDF_FLUX_G', 'MOF_PSF_FLUX_G', 'SOF_BDF_FLUX_G', 'SOF_PSF_FLUX_G',
       'MAG_AUTO_G', 'MAG_APER_4_G', 'MAG_APER_8_G', 'SOF_BDF_G_1',
       'SOF_BDF_G_2', 'IMAFLAGS_ISO_G', 'IMAFLAGS_ISO_R', 'IMAFLAGS_ISO_I',
       'IMAFLAGS_ISO_Z'],
      dtype='object')


,COADD_OBJECT_ID,TILENAME,HPIX_16384,HPIX_4096,DNF_ZMEAN_SOF,RA,DEC,FLUX_RADIUS_G,FLUX_RADIUS_R,FLUX_RADIUS_I,...,SOF_PSF_FLUX_G,MAG_AUTO_G,MAG_APER_4_G,MAG_APER_8_G,SOF_BDF_G_1,SOF_BDF_G_2,IMAFLAGS_ISO_G,IMAFLAGS_ISO_R,IMAFLAGS_ISO_I,IMAFLAGS_ISO_Z
0,1356897753,DES0259+0001,10468,654,0.119320,44.731140,0.278649,11.344465,10.402104,10.004392,...,20250.426534,17.550304,19.475899,18.317991,-0.066913,-0.019368,0,0,0,0
1,1356899604,DES0259+0001,10479,654,0.489287,44.724931,0.285954,6.793303,5.182436,4.745438,...,355.976279,22.391693,23.894903,22.665186,-0.065736,0.061260,0,0,0,0
2,1356899786,DES0259+0001,10470,654,0.114904,44.727893,0.278452,10.243550,9.611610,9.000776,...,2372.785330,19.664295,21.787668,20.332857,-0.074016,-0.782683,0,0,0,0
3,1356899808,DES0259+0001,10465,654,0.127809,44.727927,0.277062,4.426229,4.066752,3.951095,...,15632.830324,18.827084,19.775852,18.953764,-0.122955,-0.494732,0,0,0,0
4,1356899836,DES0259+0001,10477,654,0.844336,44.727754,0.285493,5.718666,7.268754,7.223648,...,243.493045,22.977860,24.335751,23.373678,-0.157839,0.060133,0,0,0,0


In [4]:
ra1 = data['RA']
dec1 = data['DE']
ra2 = catalog['RA']
dec2 = catalog['DEC']

matches = smatch.match(ra1, dec1, 0.0003, ra2, dec2, nside=4096, maxmatch=1)
data = data.drop(labels = ['RA', 'DE', 'recno', 'objID', 'HPIX_4096'], axis=1)
print(len(matches))

17205


In [5]:
new_data = pd.DataFrame()
for i in tqdm(range(len(matches))):
    #if(i == 100): break
    point = pd.concat( (data.iloc[matches[i][0]], catalog.iloc[matches[i][1]]), axis=0)
    new_data = new_data.append(point, ignore_index=True)

  0%|          | 0/17205 [00:00<?, ?it/s]

In [6]:
new_data = new_data.reindex(columns=[ 'COADD_OBJECT_ID', 'TILENAME', 'HPIX_16384', 'HPIX_4096', 'DNF_ZMEAN_SOF', 
                                     'RA', 'DEC', 'M15', 
        'Pdisk', 'Pedgeon', 'PbarGZ2', 'PbarNair10', 'Pmerg', 'Pbulge', 'Pcigar', 'TType', 'PS0', 
        'FLUX_RADIUS_G', 'FLUX_RADIUS_R', 'FLUX_RADIUS_I', 'FLUX_RADIUS_Z', 'KRON_RADIUS', 'GAP_FLUX_G',
       'MOF_BDF_FLUX_G', 'MOF_PSF_FLUX_G', 'SOF_BDF_FLUX_G', 'SOF_PSF_FLUX_G', 'MAG_AUTO_G', 'MAG_APER_4_G', 
        'MAG_APER_8_G', 'SOF_BDF_G_1', 'SOF_BDF_G_2', 'IMAFLAGS_ISO_G', 'IMAFLAGS_ISO_R', 'IMAFLAGS_ISO_I',
       'IMAFLAGS_ISO_Z'])

new_data = new_data.astype({'TILENAME': str, 'COADD_OBJECT_ID': int,'HPIX_16384': int, 'HPIX_4096': int})

In [7]:
new_data.head()

,COADD_OBJECT_ID,TILENAME,HPIX_16384,HPIX_4096,DNF_ZMEAN_SOF,RA,DEC,M15,Pdisk,Pedgeon,...,SOF_PSF_FLUX_G,MAG_AUTO_G,MAG_APER_4_G,MAG_APER_8_G,SOF_BDF_G_1,SOF_BDF_G_2,IMAFLAGS_ISO_G,IMAFLAGS_ISO_R,IMAFLAGS_ISO_I,IMAFLAGS_ISO_Z
0,1357042996,DES0259+0001,1177,73,0.044156,45.074895,0.110863,562371.0,0.007895,8.908050e-03,...,13999.692974,17.863182,19.680687,18.178492,0.270914,-0.006935,0.0,0.0,0.0,0.0
1,1357040990,DES0259+0001,1265,79,0.097686,45.090614,0.136579,562372.0,0.000452,4.362040e-07,...,7216.290993,18.199677,20.458017,18.846840,0.016191,0.090772,0.0,0.0,0.0,0.0
2,1357037576,DES0259+0001,4224,264,0.132932,45.152588,0.170296,562373.0,0.002187,9.806280e-01,...,8368.668228,18.540169,20.346113,19.096581,0.247162,0.491628,0.0,0.0,0.0,0.0
3,1356894408,DES0259+0001,7463,466,0.042477,45.215201,0.355711,149628.0,0.000280,7.518660e-02,...,7492.506393,18.239069,20.434397,18.939157,0.216854,-0.011633,0.0,0.0,0.0,0.0
4,1356899664,DES0259+0001,8568,535,0.185637,44.885581,0.231764,149625.0,0.003596,3.137700e-01,...,15574.735213,17.158770,19.785921,18.521420,-0.006813,0.192957,0.0,0.0,0.0,0.0


In [8]:
new_data.to_csv('morphological_catalog.csv', index=False)